<a href="https://colab.research.google.com/github/parkmina365/Semiconductor-defect-inspection/blob/main/yolov5_train_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv5

In [ ]:
# Git clone
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10169, done.
remote: Total 10169 (delta 0), reused 0 (delta 0), pack-reused 10169
Receiving objects: 100% (10169/10169), 10.41 MiB | 14.52 MiB/s, done.
Resolving deltas: 100% (7052/7052), done.
/content/yolov5


In [ ]:
# torch version 확인
import torch
from IPython.display import Image, clear_output  # to display images
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (Tesla P100-PCIE-16GB)


In [ ]:
# 필수 라이브러리 설치
!pip install -r /content/yolov5/requirements.txt

     |████████████████████████████████| 596 kB 4.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# roboflow를 이용하여 bounding box 처리후 url 가져오기

!curl -L "주소입력" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0    961      0 --:--:-- --:--:-- --:--:--   960
100 16.8M  100 16.8M    0     0  6785k      0  0:00:02  0:00:02 --:--:-- 42.1M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/pattern_1-1-1-35-D_jpg.rf.98b9018de36e36a17c35de39474d3d16.jpg  
 extracting: test/images/pattern_1-1-1-37-D_jpg.rf.00e086063b38e829710dfbdfec4c8bc2.jpg  
 extracting: test/images/pattern_1-1-1-39-D_jpg.rf.1c8b369b0b27876401b6ce29c0b7289b.jpg  
 extracting: test/images/pattern_1-1-1-40-D_jpg.rf.c0db016d2e7384a8fb92c63a20399ae3.jpg  
 extracting: test/images/pattern_1-1-2-10-D_jpg.rf.12594ae6dd9d02fd95d5d6f21308ac4c.jpg  
 extracting: test/images/pattern_1-1-2-12-D_jpg.rf.7

In [ ]:
# yaml 파일 경로 변경

import yaml

with open('/content/yolov5/data.yaml', 'r') as f:
  data = yaml.load(f, Loader=yaml.FullLoader)
print(data)

data['names'] = ['Bdot', 'curve', 'dot', 'straight']
data['train'] = '/content/yolov5/train'
data['val'] = '/content/yolov5/valid'
data['test'] ='/content/yolov5/test'

with open('/content/yolov5/data.yaml', 'w') as f:
  yaml.dump(data, f)
print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 4, 'names': ['Bdot', 'curve', 'dot', 'straight']}
{'train': '/content/yolov5/train', 'val': '/content/yolov5/valid', 'nc': 4, 'names': ['Bdot', 'curve', 'dot', 'straight'], 'test': '/content/yolov5/test'}


# Train
- --patience : Early Stopping patience / default=100
- --img : img size / default=640
- --batch : batch size / default=16
- --epochs : epochs / default=300
- --device : GPU 사용여부 / default=''
- --data : yaml 데이터 경로 / default=ROOT
- --cfg : config 파일 경로 / default=''
- --weights : 가중치 파일 경로 / default=ROOT
- --name : 학습 결과가 저장될 폴더 이름 / default='exp'


In [ ]:
# train
!python3 /content/yolov5/train.py --patience 0 --img 320 --batch 32 --epochs 700 --device 0 \
--data /content/yolov5/data.yaml --cfg /content/yolov5/models/yolov5m.yaml --weights /content/yolov5/yolov5m.pt --name dot_yolov5m_results

train: weights=/content/yolov5/yolov5m.pt, cfg=/content/yolov5/models/yolov5m.yaml, data=/content/yolov5/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=700, batch_size=32, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=dot_yolov5m_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=0, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-137-g2c63175 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, 

# Detect
- --img : img size / default=640
- --conf-thres : 신뢰도의 threshold / default=0.25
- --iou-thres : IOU의 threshold / default=0.45
- --device : GPU 사용여부 / default=''
- --line-thickness : 바운딩 박스의 선 두께 / default=3
- --weights : 가중치 파일 경로 / default=ROOT
- --source : 탐지할 이미지/동영상 경로 / default=ROOT

In [ ]:
# 사진 detect하기

from glob import glob
img_list = glob('/content/yolov5/valid/images/*.jpg')

for i in range(len(img_list)):
  !python3 /content/yolov5/detect.py --img 320 --conf-thres 0.5 --iou-thres 0.2 --device 0 --line-thickness 2 \
  --weights /content/yolov5/runs/train/dot_yolov5m_results/weights/best.pt --source "{img_list[i]}"

detect: weights=['/content/yolov5/runs/train/dot_yolov5m_results/weights/best.pt'], source=/content/yolov5/valid/images/pattern_2-1-2-08-D_jpg.rf.e5e0b51b7a717007397e8da64fecae37.jpg, imgsz=[320, 320], conf_thres=0.5, iou_thres=0.2, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-137-g2c63175 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 290 layers, 20865057 parameters, 0 gradients, 48.0 GFLOPs
image 1/1 /content/yolov5/valid/images/pattern_2-1-2-08-D_jpg.rf.e5e0b51b7a717007397e8da64fecae37.jpg: 320x320 Done. (0.010s)
Speed: 0.3ms pre-process, 9.5ms inference, 0.3ms NMS per image at shape (1, 3, 320, 320)
Results saved to runs/detect/exp
detect: weights=['/content/yolov5/

In [ ]:
# detect 완료된 inference 사진 다운로드

from google.colab import files

jpg_list = glob('/content/yolov5/runs/detect/*/*.jpg')
for i in range(len(jpg_list)):
  files.download(jpg_list[i])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 동영상 detect하기

%cd /content/
import gdown
file_path = '동영상경로'
output_name = 'test_video.mp4'
gdown.download(file_path, output_name, quiet=True)

!python3 /content/yolov5/detect.py --img 320 --conf-thres 0.5 --iou-thres 0.2 --device 0 --line-thickness 2 \
 --weights /content/yolov5/runs/train/dot_yolov5m_results/weights/best.pt --source /content/test_video.mp4

/content
detect: weights=['/content/yolov5/runs/train/dot_yolov5m_results/weights/best.pt'], source=/content/test_video.mp4, imgsz=[320, 320], conf_thres=0.5, iou_thres=0.2, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-137-g2c63175 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 290 layers, 20865057 parameters, 0 gradients, 48.0 GFLOPs
video 1/1 (1/2127) /content/test_video.mp4: 320x320 2 dots, Done. (0.010s)
video 1/1 (2/2127) /content/test_video.mp4: 320x320 2 dots, Done. (0.009s)
video 1/1 (3/2127) /content/test_video.mp4: 320x320 2 dots, Done. (0.009s)
video 1/1 (4/2127) /content/test_video.mp4: 320x320 2 dots, Done. (0.009s)
video 1/1 (5/2127) /content/test_v

In [ ]:
# detect 완료된 inference 동영상 다운로드

files.download(glob('/content/yolov5/runs/detect/*/*.mp4')[0])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# TensorBoard

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/train/dot_yolov5m_results